# Inferential Exclusion Prompting(IEP)

LLMに非線形な思考をさせることで精度を改善する手法．
- 論文:https://arxiv.org/abs/2310.12342

<a href="https://colab.research.google.com/github/fuyu-quant/data-science-wiki/blob/develop/nlp/llm_prompt_engineering/inferential_exclusion_prompting.ipynb" target="_blank" rel="noopener noreferrer"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from langchain.llms import OpenAI

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

llm_model = OpenAI(model_name= 'gpt-3.5-turbo', temperature=0)

/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


### 通常のプロンプト

In [25]:
prompt = """
ロジャーはテニスボールを5個持っている。彼はさらに2つのテニスボール缶を買った。それぞれの缶には3個のテニスボールが入っている。彼は今何個のテニスボールを持っていますか？
"""

In [26]:
output = llm_model(prompt)
print(output)

ロジャーは最初に5個のテニスボールを持っていました。さらに2つのテニスボール缶を買ったので、それぞれの缶には3個のテニスボールが入っています。したがって、2つの缶から合計で2 * 3 = 6個のテニスボールを追加で手に入れました。したがって、ロジャーは最終的に5 + 6 = 11個のテニスボールを持っています。


### Inferential Exclusion Prompting

* 回答をいくつか出力

In [2]:
question = """
ロジャーはテニスボールを5個持っている。彼はさらに2つのテニスボール缶を買った。それぞれの缶には3個のテニスボールが入っている。彼は今何個のテニスボールを持っていますか？
"""

In [7]:
prompt1 = """
Q:{question_}
上記の質問に対する回答として考えられるものを３つ出力してください．
""".format(question_=question)

In [8]:
output1 = llm_model(prompt1)
print(output1)

1. ロジャーは最初に持っていたテニスボールの数は5個であり、さらに2つのテニスボール缶を買ったので、合計で5 + 2 × 3 = 11個のテニスボールを持っています。

2. ロジャーは最初に持っていたテニスボールの数は5個であり、1つのテニスボール缶には3個のテニスボールが入っているので、2つの缶で合計で5 + 2 × 3 = 11個のテニスボールを持っています。

3. ロジャーは最初に持っていたテニスボールの数は5個であり、2つのテニスボール缶を買ったので、合計で5 + 2 × 3 = 11個のテニスボールを持っています。


In [9]:
answer_1 = """
ロジャーは最初に持っていたテニスボールの数は5個であり、さらに2つのテニスボール缶を買ったので、合計で5 + 2 × 3 = 11個のテニスボールを持っています。
"""

answer_2 = """
ロジャーは最初に持っていたテニスボールの数は5個であり、1つのテニスボール缶には3個のテニスボールが入っているので、2つの缶で合計で5 + 2 × 3 = 11個のテニスボールを持っています。
"""

answer_3 = """
ロジャーは最初に持っていたテニスボールの数は5個であり、2つのテニスボール缶を買ったので、合計で5 + 2 × 3 = 11個のテニスボールを持っています。
"""


* 前提の確認

In [19]:
prompt2_1 = """
Q:{question_}
A:{answer_1}
上記の回答が正しい前提はなんですか?
""".format(question_=question, answer_1=answer_1)

output2_1 = llm_model(prompt2_1)
print(output2_1)

上記の回答が正しい前提は、テニスボール缶1つに3個のテニスボールが入っているということです。


In [20]:
prompt2_2 = """
Q:{question_}
A:{answer_2}
上記の回答が正しい前提はなんですか?
""".format(question_=question, answer_2=answer_2)

output2_2 = llm_model(prompt2_2)
print(output2_2)

上記の回答が正しい前提は、ロジャーが最初に持っていたテニスボールの数が5個であり、1つのテニスボール缶には3個のテニスボールが入っていることです。また、ロジャーが2つのテニスボール缶を買ったという情報も正しい前提です。


In [21]:
prompt2_3 = """
Q:{question_}
A:{answer_3}
上記の回答が正しい前提はなんですか?
""".format(question_=question, answer_3=answer_3)

output2_3 = llm_model(prompt2_3)
print(output2_3)

上記の回答が正しい前提は、テニスボール缶のそれぞれに3個のテニスボールが入っているということです。


* 答えの確認

In [22]:
prompt3_1 = """
Q:{question_}
A:{answer_1}
前提:{output2_1}
上記の回答を仮説として考えると，前提に合致していますか?
""".format(question_=question, answer_1=answer_1, output2_1=output2_1)

output3_1 = llm_model(prompt3_1)
print(output3_1)

はい、上記の回答は仮説として考えると、前提に合致しています。


In [23]:
prompt3_2 = """
Q:{question_}
A:{answer_2}
前提:{output2_2}
上記の回答を仮説として考えると，前提に合致していますか?
""".format(question_=question, answer_2=answer_2, output2_2=output2_2)

output = llm_model(prompt3_2)
print(output)

はい、上記の回答は前提に合致しています。ロジャーが最初に持っていたテニスボールの数が5個であり、1つのテニスボール缶には3個のテニスボールが入っているという情報が与えられており、またロジャーが2つのテニスボール缶を買ったという情報も与えられています。したがって、ロジャーは合計で11個のテニスボールを持っているという結論が導かれます。


In [24]:
prompt3_3 = """
Q:{question_}
A:{answer_3}
前提:{output2_3}
上記の回答を仮説として考えると，前提に合致していますか?
""".format(question_=question, answer_3=answer_3, output2_3=output2_3)

output3_3 = llm_model(prompt3_3)
print(output3_3)

はい、上記の回答は仮説として考えると、前提に合致しています。
